In [ ]:
# # using groupby
# very_important_features = ['chocolate', 'peanutyalmondy', 'caramel']
# relation = data.groupby(very_important_features).agg({'winpercent':'mean'}).reset_index().rename(columns={'winpercent':'winpercent_mean'})
# winpercent_count = data.groupby(very_important_features).agg({'winpercent':'count'}).reset_index().rename(columns={'winpercent':'winpercent_count'}).winpercent_count
# relation['winpercent_count'] = winpercent_count
# relation.sort_values(by=['winpercent_mean','winpercent_count'], ascending=False, inplace=True)
# relation

In [ ]:
pca = PCA(random_state=1)
pca.fit_transform(data.iloc[:,1:])

df_pca = pd.DataFrame({'components':range(1,len(data.columns[1:])+1), \
    'explained variance ratio':pca.explained_variance_ratio_, 'cumulative explained variance ratio':np.cumsum(pca.explained_variance_ratio_)})
df_pca


fig = go.Figure()
fig = make_subplots(rows=1, cols=2)
fig.add_trace(go.Scatter(x=df_pca.components, y=df_pca['explained variance ratio'], mode='lines+markers'), row=1, col=1)
fig.add_trace(go.Scatter(x=df_pca.components, y=df_pca['cumulative explained variance ratio'], mode='lines+markers'), row=1, col=2)
fig.update_xaxes(title_text="principal components", row=1, col=1)
fig.update_yaxes(title_text="explained variance ratio", row=1, col=1)
fig.update_xaxes(title_text="number of components", row=1, col=2)
fig.update_yaxes(title_text="cumulative explained variance ratio", row=1, col=2)
fig.update_layout(title_text='PCA', title_font=dict(size=29, color='#8a8d93', family="Lato, sans-serif"), showlegend=False)
fig.show()

# 2d
comp2 = pd.DataFrame({'PC1':pca.components_[0],'PC2':pca.components_[1], 'features':data.iloc[:,1:].columns})
fig = plt.figure(figsize = (6,6))
px.scatter(comp2, x='PC1', y='PC2', color='features', text="features")


# 3d
pca = PCA(n_components=3)
components = pca.fit_transform(X)

total_var = pca.explained_variance_ratio_.sum() * 100

fig = px.scatter_3d(
    components, x=0, y=1, z=2, color=df['species'],
    title=f'Total Explained Variance: {total_var:.2f}%',
    labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'}
)
fig.show()

### Logistic regression

In [ ]:
features = data.columns[2:]
X_train = data[features]
y_train = data.chocolate.ravel()

random_grid = {'C':np.logspace(-4, 4, 50), 'solver':['newton-cg','lbfgs','liblinear','sag','saga'], 'max_iter':[300,500,700]}

lr = LogisticRegression()
model_lr = RandomizedSearchCV(estimator = lr, param_distributions = random_grid, cv = 4, refit = True, random_state=42, n_jobs = 2)
model_lr.fit(X_train, y_train)

pred_lr = cross_val_predict(model_lr.best_estimator_, X_train, y_train, cv=4, n_jobs=2)

print(f'model_rfg best parameters: {model_lr.best_params_}, \nR2 score: {metrics.r2_score(y_train, pred_lr)}, \
    \nRoot Mean Square Error: {math.sqrt(metrics.mean_squared_error(y_train, pred_lr))}')

df_lr = pd.DataFrame({'feature': features, 'importance': model_lr.best_estimator_.coef_.squeeze()})
px.bar(df_lr, x='feature', y='importance').update_xaxes(categoryorder="total descending")

Some of the interesting conclusions about this analysis:

1) __Chocolate__ is the most effective component which affects the popularity. In caontrary, __Fruity__ goes in a completly different direction as others.

2) __Do not make fruit candies or hard candies packt in a bag__ as much as possible.

3) Candies containing __chocolate and caramel or peanutyalmondy__ in the forms __bar or crispedricewafer__ are more probable to sell.

4) If we want the candies that has __more over 55% winpercentage__, we have to make soft candies contain __chocolate and caramel__.

5) If we want the candies that has __more over 60% winpercentage__, we have to make soft bar candies contain __chocolate and wafer__. (Then, select one from peanutyalmondy and caramel or both.)

7) __Recommendotion__: Soft bar candy with chocolate and without fruit, separate packing, select peanutyalmondy or crispedricewafer or caramel one or more. The price can be increased to more than 60 percent.

In [ ]:
print('soft bar candies contain chocolate and wafer')
sort_data.query('chocolate==1 & crispedricewafer==1 & bar==1')  #  & fruity==0 & hard==0